In [2]:
from __future__ import division
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib inline
from sklearn.decomposition import PCA
import pandas as pd
import math
from numpy import random
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score
from imblearn.metrics import geometric_mean_score
from sklearn.metrics import f1_score

In [3]:
df = pd.read_csv('../data/CTGsmt.csv', header=None)
data = df.to_numpy()
# Features matrix
X = data[:,0:22]
Y = data[:,23]

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=0, stratify=Y, shuffle=True)

In [12]:
def prueba_svm(ker='linear',conf=0.001,gam=0.0):
    Folds = 4
    random.seed(19680801)
    EficienciaTrain = np.zeros(Folds)
    EficienciaVal = np.zeros(Folds)
    skf = StratifiedKFold(n_splits=Folds)
    porc_vectores = np.zeros(Folds)
    f1 = np.zeros(Folds)
    j = 0
    gmean = np.zeros(Folds)
    for train, test in skf.split(X, Y):
        Xtrain = X[train,:]
        Ytrain = Y[train]
        Xtest = X[test,:]
        Ytest = Y[test]
        
        #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
        if ker == 'linear':
            modelo = SVC(kernel =ker, C=conf)
        else:
            modelo = SVC(kernel=ker, C= conf, gamma= gam)
        modelo.fit(Xtrain,Ytrain)


        #Validación
        Ytrain_pred = modelo.predict(Xtrain)
        Yest = modelo.predict(Xtest)

        #Evaluamos las predicciones del modelo con los datos de test      
        gmean[j] =  geometric_mean_score(y_true = Ytest, y_pred=Yest, average="weighted")
        f1[j] = f1_score(y_true = Ytest, y_pred=Yest, average = "weighted")
        EficienciaTrain[j] = np.mean(Ytrain_pred.ravel() == Ytrain.ravel())
        EficienciaVal[j] = np.mean(Yest.ravel() == Ytest.ravel())
        porc_vectores[j] = len(modelo.support_vectors_)/len(Xtrain)
        j += 1

    f1_m = np.mean(f1)
    f1_ic = np.std(f1)
    gmean = np.mean(gmean)
    gmean_ic = np.std(gmean)
    efi_train = np.mean(EficienciaTrain)
    efi_train_ic = np.std(EficienciaTrain)
    efi_test = np.mean(EficienciaVal)
    efi_test_ic = np.std(EficienciaVal)

    print('f1 = ' + str(f1_m) + '+-' + str(f1_ic))
    print('gmean = ' + str(gmean) + '+-' + str(gmean_ic))
    print('Eficiencia Train= ' + str(efi_train) + '+-' + str(efi_train_ic))
    print('Eficiencia Test= ' + str(efi_test) + '+-' + str(efi_test_ic))
        
    return f1_m, f1_ic, gmean, gmean_ic,efi_train, efi_train_ic, efi_test, efi_test_ic, porc_vectores

In [ ]:
import pandas as pd
import qgrid
df_types = pd.DataFrame({
    'Kernel' : pd.Series(['linear','linear','linear','linear','linear','linear','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf']),
    'C' : pd.Series([0.001,0.01,0.1,1,10,100,0.001,0.001,0.001,0.01,0.01,0.01,0.1,0.1,0.1,1,1,1,10,10,10,100,100,100]),
    'gamma' : pd.Series([0,0,0,0,0,0,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1])})

df_types["f1"] = ""
df_types["IC f1"] = ""
df_types["gmean"] = ""
df_types["IC gmean"] = ""
df_types["Eficiencia Train"] = ""
df_types["IC Eficiencia Train"] = ""
df_types["Eficiencia Test"] = ""
df_types["IC Eficiencia Test"] = ""
df_types["% de Vectores de Soporte"] = ""
df_types.set_index(['Kernel','C','gamma'], inplace=True)

In [ ]:
j = 0
for i in df_types.index:
    print(i)
    kernel = i[0]
    c = i[1]
    gamma = i[2]
    f1_m, f1_ic, gmean, gmean_ic, efi_train, efi_train_ic, efi_test, efi_test_ic, porc_vectore = prueba_svm(ker=kernel,conf=c,gam=gamma)
    df_types["f1"] = str(f1_m)
    df_types["IC f1"] = str(f1_ic)
    df_types["gmean"] = str(gmean)
    df_types["IC gmean"] = str(gmean_ic)
    df_types["Eficiencia Train"] = efi_train
    df_types["IC Eficiencia Train"] = efi_train_ic
    df_types["Eficiencia Test"] = efi_test
    df_types["IC Eficiencia Test"] = efi_test_ic
    df_types["% de Vectores de Soporte"] = str(porc_vectore)
    j = j+1

In [15]:
def train_svm(kernel, C, gamma):
    Folds = 4
    random.seed(19680801)
    f1 = np.zeros(Folds)
    gmean = np.zeros(Folds)
    skf = StratifiedKFold(n_splits=Folds)
    EficienciaTrain = np.zeros(Folds)
    EficienciaVal = np.zeros(Folds)
    porc_vectores = np.zeros(Folds)
    j = 0
    models = []
    for train, test in skf.split(X, Y):
        Xtrain = X[train,:]
        Ytrain = Y[train]
        Xtest = X[test,:]
        Ytest = Y[test]

        #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
        model = SVC(C=C,kernel=kernel, gamma=gamma, probability=True)
        model.fit(Xtrain, Ytrain)
        models.append(model)
        #Validación
        Y_pred=model.predict_proba(Xtest)
        Ytrain_pred = model.predict(Xtrain)#Use el modelo previamente entrenado para hacer predicciones con las mismas muestras de entrenamiento
        Yest = model.predict(Xtest)#Use el modelo previamente entrenado para hacer predicciones con las muestras de test

        #Evaluamos las predicciones del modelo con los datos de test

     #Evaluamos las predicciones del modelo con los datos de test      
        gmean[j] =  geometric_mean_score(y_true = Ytest, y_pred=Yest, average="weighted")
        f1[j] = f1_score(y_true = Ytest, y_pred=Yest, average = "weighted")
        EficienciaTrain[j] = np.mean(Ytrain_pred.ravel() == Ytrain.ravel())
        EficienciaVal[j] = np.mean(Yest.ravel() == Ytest.ravel())
        porc_vectores[j] = len(model.support_vectors_)/len(Xtrain)
        j += 1

    f1_m = np.mean(f1)
    f1_ic = np.std(f1)
    gmean = np.mean(gmean)
    gmean_ic = np.std(gmean)
    efi_train = np.mean(EficienciaTrain)
    efi_train_ic = np.std(EficienciaTrain)
    efi_test = np.mean(EficienciaVal)
    efi_test_ic = np.std(EficienciaVal)

    print('f1 = ' + str(f1_m) + '+-' + str(f1_ic))
    print('gmean = ' + str(gmean) + '+-' + str(gmean_ic))
    print('Eficiencia Train= ' + str(efi_train) + '+-' + str(efi_train_ic))
    print('Eficiencia Test= ' + str(efi_test) + '+-' + str(efi_test_ic))
        
    return f1_m, f1_ic, gmean, gmean_ic,efi_train, efi_train_ic, efi_test, efi_test_ic, porc_vectores

In [ ]:
df_types = pd.DataFrame({
    'Kernel' : pd.Series(['lineal','lineal','lineal','lineal','lineal','lineal','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf']),
    'C' : pd.Series([0.001,0.01,0.1,1,10,100,0.001,0.001,0.001,0.01,0.01,0.01,0.1,0.1,0.1,1,1,1,10,10,10,100,100,100]),
    'gamma' : pd.Series([0,0,0,0,0,0,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1])})

df_types.set_index(['Kernel','C','gamma'], inplace=True)


In [16]:
import qgrid
randn = np.random.randn

f1_ = []
stdf1_ = []
gmean_ = []
stdgmean_ = []
efi_train_ = []
std_efi_train_ = []
efi_test_ = []
std_efi_test_ = []
for e, i, j in zip(df_types['Kernel'], df_types['C'], df_types['gamma']):
    if(e == 'lineal'):
        f1_m, f1_ic, gmean, gmean_ic, efi_train, efi_train_ic, efi_test, efi_test_ic, porc_vectores = train_svm("linear", i, "auto")
    else:
        f1_m, f1_ic, gmean, gmean_ic, efi_train, efi_train_ic, efi_test, efi_test_ic, porc_vectores = train_svm(e, i, j)
    f1_.append(f1_m)
    stdf1_.append(f1_ic)
    gmean_.append(gmean)
    stdgmean_.append(gmean_ic)
    efi_train_.append(efi_train)
    std_efi_train_.append(efi_train_ic)
    efi_test_.append(efi_test)
    std_efi_test_.append(efi_test_ic)
    

df_types["f1"] = f1_
df_types["Intervalo de confianza f1"] = stdf1_
df_types["G-mean"] = gmean_
df_types["Intervalo de confianza G-mean"] = stdgmean_
df_types["Eficiencia Train"] = efi_train_
df_types["IC Eficiencia Train"] = std_efi_train_
df_types["Eficiencia Test"] = efi_test_
df_types["IC Eficiencia Test"] = std_efi_test_

f1 = 0.5802298448636799+-0.0
gmean = 0.7069487321952701+-0.0
Eficiencia Train= 0.6170598736971357+-0.07425919864572549
Eficiencia Test= 0.6189042588115916+-0.10275617353048849
f1 = 0.7818783851459411+-0.0
gmean = 0.8334832366290696+-0.0
Eficiencia Train= 0.7993289961983352+-0.007120904592584718
Eficiencia Test= 0.780459728726379+-0.03251717343629171
f1 = 0.8306485353087093+-0.0
gmean = 0.8715506901078198+-0.0
Eficiencia Train= 0.8561265710321619+-0.0033681054082311643
Eficiencia Test= 0.8302111758607221+-0.039371808182593344
f1 = 0.8515561007312563+-0.0
gmean = 0.8873902610779109+-0.0
Eficiencia Train= 0.8739176095816841+-0.0047477593461559674
Eficiencia Test= 0.8509591117235723+-0.033808134639144345
f1 = 0.8700425955055768+-0.0
gmean = 0.9015052428762647+-0.0
Eficiencia Train= 0.8924475341686784+-0.003997465453927471
Eficiencia Test= 0.8694902817191994+-0.023126750985295094
f1 = 0.8687095377863748+-0.0
gmean = 0.9005803526339351+-0.0
Eficiencia Train= 0.894797338546089+-0.004220388900

In [17]:
df_types

f1  Intervalo de confianza f1    G-mean  \
Kernel C       gamma                                                  
lineal 0.001   0.00   0.580230                        0.0  0.706949   
       0.010   0.00   0.781878                        0.0  0.833483   
       0.100   0.00   0.830649                        0.0  0.871551   
       1.000   0.00   0.851556                        0.0  0.887390   
       10.000  0.00   0.870043                        0.0  0.901505   
       100.000 0.00   0.868710                        0.0  0.900580   
rbf    0.001   0.01   0.315290                        0.0  0.557719   
               0.10   0.315414                        0.0  0.557874   
               1.00   0.313692                        0.0  0.556788   
       0.010   0.01   0.315290                        0.0  0.557719   
               0.10   0.719163                        0.0  0.784949   
               1.00   0.815892                        0.0  0.859753   
       0.100   0.01   0.722851                        0.0  0.787749   
               0.10   0.809233                        0.0  0.854926   
               1.00   0.854807                        0.0  0.890138   
       1.000   0.01   0.808141                        0.0  0.854157   
               0.10   0.846052                        0.0  0.883517   
               1.00   0.910729                        0.0  0.933400   
       10.000  0.01   0.834540                        0.0  0.874613   
               0.10   0.882449                        0.0  0.911122   
               1.00   0.927811                        0.0  0.946486   
       100.000 0.01   0.867877                        0.0  0.899819   
               0.10   0.915680                        0.0  0.936761   
               1.00   0.942724                        0.0  0.957872   

                      Intervalo de confianza G-mean  Eficiencia Train  \
Kernel C       gamma                                                    
lineal 0.001   0.00                             0.0          0.617060   
       0.010   0.00                             0.0          0.799329   
       0.100   0.00                             0.0          0.856127   
       1.000   0.00                             0.0          0.873918   
       10.000  0.00                             0.0          0.892448   
       100.000 0.00                             0.0          0.894797   
rbf    0.001   0.01                             0.0          0.431574   
               0.10                             0.0          0.433857   
               1.00                             0.0          0.436745   
       0.010   0.01                             0.0          0.431574   
               0.10                             0.0          0.730850   
               1.00                             0.0          0.831219   
       0.100   0.01                             0.0          0.730984   
               0.10                             0.0          0.834710   
               1.00                             0.0          0.882847   
       1.000   0.01                             0.0          0.832360   
               0.10                             0.0          0.871031   
               1.00                             0.0          0.941256   
       10.000  0.01                             0.0          0.860289   
               0.10                             0.0          0.908561   
               1.00                             0.0          0.969184   
       100.000 0.01                             0.0          0.889896   
               0.10                             0.0          0.942196   
               1.00                             0.0          0.986841   

                      IC Eficiencia Train  Eficiencia Test  IC Eficiencia Test  
Kernel C       gamma                                                            
lineal 0.001   0.00              0.074259         0.618904            0.102756  
       0.010   0.00              0.0071